# Notebook to Construct an Augmented dG databse using a trained model

Author: Wylie Kau, Last Edit: 2/22/2025

v3

Notes
- Built support for various model types, unique pathing to final datasets.

## Imports

In [1]:
# imports
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import seaborn as sns
import numpy as np
import pandas as pd
import rdkit
from rdkit.Chem import Draw, Lipinski, Crippen, Descriptors
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG
import pubchempy as pcp
import re
from datetime import datetime
import pickle
import traceback
import os

plt.style.use('wfk')

### Data Imports

Import the prepared dataset with feature and target

In [2]:
df = pd.read_csv("../Outputs/Featurized NIST46 Dataset for Model Training/AllMolEncodings+set5-2_ion-encodings.csv")
display(df.head())

non_features = ['delta_G', 'smiles', 'Electrophile', 'Ligand']
features = [f for f in list(df.columns) if f not in non_features]
target = ['delta_G']

ligands = list(df['smiles'].unique())
ephiles = list(df['Electrophile'].unique())
print(ephiles)
max_l_e_pairs = len(ligands) * len(ephiles)

ligands = list(df['smiles'].unique())
ligand_to_index_map = df.groupby('smiles').apply(lambda x: x.index.tolist()).to_dict()
ephile_to_index_map = df.groupby('Electrophile').apply(lambda x: x.index.tolist()).to_dict()
ephile_to_ligand_map = df.groupby('Electrophile').apply(lambda x: list(x['smiles'].unique())).to_dict()

print(f"{len(ligands)} Ligands, {len(ephiles)} Electrophiles")
unique_l_e_pairs = df[['smiles', 'Electrophile']].drop_duplicates().shape[0]
print(f"Number of observed unique ligand-electrophile pairs: {unique_l_e_pairs}")
print(f"Max Possible Ligand-Electrophile Pairs: {max_l_e_pairs}, {unique_l_e_pairs/max_l_e_pairs*100:.2f}% of max possible pairs")
print(f'Training Dataset: {df.shape[0]} Datapoints, {len(features)} Features')

print(f'{len(features)} total features')

print()

,Ligand,Electrophile,smiles,delta_G,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,...,Ion-Water Internuclear Distance [nm],Ionic Radii in Solution [nm],Pauling Crystal Ionic Radii [nm],1st Ionization Energy [eV],2nd Ionization Energy [eV],3rd Ionization Energy [eV],4th Ionization Energy [eV],5th Ionization Energy [eV],Standard Reduction Potentials to Neutral [V],Pauling Electronegativity
0,Aminoacetic acid (Glycine),Mg2+,NCC(=O)O,-0.300251,9.243056,9.243056,0.277778,-0.967593,0.421171,7.4,...,0.209,0.07,0.072,7.6462,15.03527,80.1437,109.2655,141.27,-2.372,1.31
1,Aminoacetic acid (Glycine),Mg2+,NCC(=O)O,-0.173385,9.243056,9.243056,0.277778,-0.967593,0.421171,7.4,...,0.209,0.07,0.072,7.6462,15.03527,80.1437,109.2655,141.27,-2.372,1.31
2,Aminoacetic acid (Glycine),Mg2+,NCC(=O)O,-0.152824,9.243056,9.243056,0.277778,-0.967593,0.421171,7.4,...,0.209,0.07,0.072,7.6462,15.03527,80.1437,109.2655,141.27,-2.372,1.31
3,Aminoacetic acid (Glycine),Mg2+,NCC(=O)O,-0.251939,9.243056,9.243056,0.277778,-0.967593,0.421171,7.4,...,0.209,0.07,0.072,7.6462,15.03527,80.1437,109.2655,141.27,-2.372,1.31
4,Aminoacetic acid (Glycine),Mg2+,NCC(=O)O,-0.433872,9.243056,9.243056,0.277778,-0.967593,0.421171,7.4,...,0.209,0.07,0.072,7.6462,15.03527,80.1437,109.2655,141.27,-2.372,1.31


['Mg2+', 'Ca2+', 'Sr2+', 'La3+', 'Pr3+', 'Nd3+', 'Sm3+', 'Gd3+', 'Tb3+', 'Dy3+', 'Er3+', 'Mn2+', 'Co2+', 'Ni2+', 'Cu2+', 'Al3+', 'Zn2+', 'Na+', 'Fe3+', 'Li+', 'K+', 'Rb+', 'Cs+']
1091 Ligands, 23 Electrophiles
Number of observed unique ligand-electrophile pairs: 4949
Max Possible Ligand-Electrophile Pairs: 25093, 19.72% of max possible pairs
Training Dataset: 7007 Datapoints, 207 Features
207 total features



/var/folders/51/1469qh3n09s2ntzgsg9vcfzc0000gn/T/ipykernel_75953/436105554.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ligand_to_index_map = df.groupby('smiles').apply(lambda x: x.index.tolist()).to_dict()
/var/folders/51/1469qh3n09s2ntzgsg9vcfzc0000gn/T/ipykernel_75953/436105554.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ephile_to_index_map = df.groupby('Electrophile').apply(lambda x: x.

In [3]:
# import electrophile dataframe
path_ephiles = '../Data/ephile properties_cleaned.xlsx'
df_e = pd.read_excel(path_ephiles, sheet_name='Set5')
#display(df_e)

df_e = df_e[df_e['Electrophile'].isin(ephiles)]
df_e = df_e.dropna(axis=1, how='all')

print("Electrophile Properties Dataset")
display(df_e.head())
#print(df_e.columns)

ephiles = df_e['Electrophile'].unique()
print("Electrophiles = ", ephiles)

Electrophile Properties Dataset


,Electrophile,Atomic Number,Ephile Molecular Weight,Charge,Single Ion Hydration Enthalpy [kcal/mol],Single Ion Hydration Gibbs Free Energy [kcal/mol],Single Ion Hydration Entropy-25C [cal/(deg*mol)],Hydrated Cation Standard Partial Molal Hydrated Entropy [cal/(deg*mol)],TdS_Lattice to Solution-25C [kcal/mol],dS_tr_1mtoH2O [cal/(deg*mol)],...,Ion-Water Internuclear Distance [nm],Ionic Radii in Solution [nm],Pauling Crystal Ionic Radii [nm],1st Ionization Energy [eV],2nd Ionization Energy [eV],3rd Ionization Energy [eV],4th Ionization Energy [eV],5th Ionization Energy [eV],Standard Reduction Potentials to Neutral [V],Pauling Electronegativity
0,Li+,3,6.940,1,-123.0,-122.1,-28.4,2.7,-1.4,-15.7,...,0.2080,0.071,0.074,5.3917,75.64000,122.45429,1000000.0000,1000000.000,-3.040,0.98
2,Na+,11,22.990,1,-96.9,-98.2,-20.9,14.0,1.0,-7.6,...,0.2356,0.097,0.102,5.1391,47.28640,71.62000,98.9100,138.400,-2.710,0.93
3,Mg2+,12,24.305,2,-459.4,-455.5,-64.0,-33.0,-11.5,-49.6,...,0.2090,0.070,0.072,7.6462,15.03527,80.14370,109.2655,141.270,-2.372,1.31
4,Al3+,13,26.982,3,-1113.7,-1103.3,-111.0,-76.9,-27.1,-101.9,...,0.1887,0.050,0.053,5.9858,18.82855,28.44765,119.9920,153.825,-1.662,1.61
5,K+,19,39.098,1,-76.7,-80.6,-12.4,24.1,3.2,-0.3,...,0.2798,0.141,0.138,4.3407,31.63000,45.80600,60.9100,82.660,-2.931,0.82


Electrophiles =  ['Li+' 'Na+' 'Mg2+' 'Al3+' 'K+' 'Ca2+' 'Mn2+' 'Fe3+' 'Co2+' 'Ni2+' 'Cu2+'
 'Zn2+' 'Rb+' 'Sr2+' 'Cs+' 'La3+' 'Pr3+' 'Nd3+' 'Sm3+' 'Gd3+' 'Tb3+'
 'Dy3+' 'Er3+']


In [4]:
ep_features = [i for i in df_e.columns if i not in ['Electrophile']]
ep_features = [i for i in ep_features if i in features]
#print(ep_features)
l_features = [i for i in features if i not in ep_features]
#print(l_features)

print(f'{len(ep_features)} Electrophile Features, {len(l_features)} Ligand Features, {len(ep_features) + len(l_features)} Total Features')

16 Electrophile Features, 191 Ligand Features, 207 Total Features


### 2.1 - dG Constructed with Trained XGBoost Regressor

In [5]:
# Import trained model with predicted error details
model_dir = '../Outputs/Fitted Models/XGBoost_HalvingGridSearchCV_20250313_163658'
model_path = os.path.join(model_dir, 'model.pkl')
error_df_path = os.path.join(model_dir, 'error_df.csv')

with open(model_path, 'rb') as file:
    model = pickle.load(file)

model_class = model.__class__.__name__
print(f"Model Class: {model_class}")

# Load error_df
error_df = pd.read_csv(error_df_path)
display(error_df)

# Convert error_df into an error dictionary
error_dict = error_df.set_index('Error Category')['RMSE [kcal/mol]'].to_dict()

Model Class: XGBRegressor


,Error Category,RMSE [kcal/mol]
0,Mg2+,0.170554
1,Ca2+,0.259343
2,Sr2+,0.173171
3,La3+,0.117529
4,Pr3+,0.086776
5,Nd3+,0.079208
6,Sm3+,0.124234
7,Gd3+,0.104659
8,Tb3+,0.065223
9,Dy3+,0.066843


In [6]:
# generate constructed dG database with error.
df_dG = pd.DataFrame()

training_ligands = ligands

error_ligands = []

for i, l in enumerate(training_ligands):
    df_dG.loc[i, 'smiles'] = l
    for e in ephiles:
        # 1 - check if ligand-ion interaction is reported in dataset
        df_obs = df[(df['smiles'] == l)]
        df_obs = df_obs[(df_obs['Electrophile'] == e)]

        observed = None

        # 2 - If yes, observed, report mean and standard for observations.
        if df_obs.shape[0] > 0:
            observed = True
            dG_mean = df_obs['delta_G'].mean()
            dG_std = df_obs['delta_G'].std()
        
        # 3 - If no, predict dG using model, extracting e-phile features from df_e for the electrophile of interest
        else:
            try:
                observed = False
                # extract ligand related features
                ligand_specific_features = df[df['smiles'] == l].loc[:, l_features].iloc[0].values

                # extract electrophile related features
                ephile_specific_features = df_e[df_e['Electrophile'] == e][ep_features].values

                # concatenate features for input datarow
                input_features = np.concatenate((ligand_specific_features, ephile_specific_features.flatten())).reshape(1, -1)  
                
                # predict dG_mean
                dG_mean = model.predict(input_features)[0]
                dG_std = error_dict[e]
                
            except:
                traceback.print_exc()
                error_ligands.append((l, e))
                continue
            
        # 4 - Save results to df_dG
        df_dG.loc[i, e+' observed'] = observed
        df_dG.loc[i, e+' dG_mean [kcal/mol]'] = float(dG_mean)
        df_dG.loc[i, e+' dG_std [kcal/mol]'] = float(dG_std)
        
    #break

# for datapoints that are observed BUT have a single observation, set the stddev to be the mean stddev observed
dG_std_observed_all = np.array([])

for e in ephiles:
    observed_col = f'{e} observed'
    dG_std_col = f'{e} dG_std [kcal/mol]'
    
    # Filter the rows where the observed value is True and dG_std is not NaN
    observed_dG_std = df_dG[(df_dG[observed_col] == True) & (df_dG[dG_std_col].notna())][dG_std_col]
    
    dG_std_observed_all = np.concatenate((dG_std_observed_all, observed_dG_std.values))

# Calculate the mean standard deviation for the observed values
mean_dG_std = round(np.mean(dG_std_observed_all), 4)
print(f"Mean standard deviation on observed dG_std data across all ions: {mean_dG_std} [kcal/mol]")

for e in ephiles:
    # Identify the columns for observed and dG_std
    observed_col = f'{e} observed'
    dG_std_col = f'{e} dG_std [kcal/mol]'
    
    # Assign the average standard deviation to NaN values where the interaction is observed
    df_dG.loc[(df_dG[observed_col] == True) & (df_dG[dG_std_col].isna()), dG_std_col] = mean_dG_std

display(df_dG.head())

Mean standard deviation on observed dG_std data across all ions: 0.0773 [kcal/mol]


,smiles,Li+ observed,Li+ dG_mean [kcal/mol],Li+ dG_std [kcal/mol],Na+ observed,Na+ dG_mean [kcal/mol],Na+ dG_std [kcal/mol],Mg2+ observed,Mg2+ dG_mean [kcal/mol],Mg2+ dG_std [kcal/mol],...,Gd3+ dG_std [kcal/mol],Tb3+ observed,Tb3+ dG_mean [kcal/mol],Tb3+ dG_std [kcal/mol],Dy3+ observed,Dy3+ dG_mean [kcal/mol],Dy3+ dG_std [kcal/mol],Er3+ observed,Er3+ dG_mean [kcal/mol],Er3+ dG_std [kcal/mol]
0,NCC(=O)O,False,0.635399,0.228217,False,1.014721,0.349653,True,-0.262454,0.112778,...,0.077300,True,-0.758856,0.077300,True,-0.758856,0.077300,True,-0.775088,0.077300
1,C[C@H](N)C(=O)O,False,0.565267,0.228217,False,0.837542,0.349653,True,-0.398668,0.077300,...,0.104659,False,-0.949830,0.065223,False,-0.961139,0.066843,False,-0.963723,0.093821
2,O=C(O)[C@H](CS)NCCN[C@@H](CS)C(=O)O,False,-0.573419,0.228217,False,-0.113043,0.349653,False,-0.938122,0.170554,...,0.104659,False,-1.567909,0.065223,False,-1.571129,0.066843,False,-1.572623,0.093821
3,CC(=O)[C@H]1CC[C@@]2(O)[C@@H]3CCC4=CC(=O)CC[C@...,False,-0.613463,0.228217,False,-0.365220,0.349653,False,-1.322750,0.170554,...,0.077300,False,-1.837327,0.065223,False,-1.833348,0.066843,False,-1.826532,0.093821
4,CC(O)CN1CCN(CC(=O)O)CCN(CC(=O)O)CCN(CC(=O)O)CC1,False,-0.620767,0.228217,False,-0.085515,0.349653,False,-1.271448,0.170554,...,0.077300,False,-1.796955,0.065223,False,-1.797202,0.066843,False,-1.798289,0.093821


In [ ]:
save_dG_constructed = False

dir_path = '../Outputs/Augmented NIST46 Dataset/'
dataset_name = f'{model_class}_dGConstructed_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'

if save_dG_constructed:
    full_path = os.path.join(dir_path, dataset_name)
    df_dG.to_csv(full_path, index=False)
